<a href="https://colab.research.google.com/github/ShyamRajana05/Infosys_Text_Summarizer/blob/main/Abs_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Set up environment and install requirements
!pip install transformers PyPDF2 spacy torch streamlit pyngrok

# Import necessary libraries
import spacy
from transformers import pipeline
import PyPDF2
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import streamlit as st
from pyngrok import ngrok
import os
import multiprocessing
import time


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Load SpaCy model for English
nlp = spacy.load("en_core_web_sm")

# Step 3: Initialize the summarization pipeline using a pre-trained model
def init_pipeline():
    try:
        # Specify the model explicitly
        summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
    except RuntimeError as e:
        return str(e)
    return summarizer

summarizer = init_pipeline()

if isinstance(summarizer, str):
    print('Error Initializing Pipeline:', summarizer)
else:
    print('Pipeline Initialized Successfully')

# Step 4: Function to summarize text using a hybrid approach with SpaCy
def hybrid_summarize(text):
    doc = nlp(text)
    sentences = list(doc.sents)

    if not sentences:
        return "No content to summarize."

    # Extractive summarization: Select the top 5 longest sentences
    extracted_sentences = sorted(sentences, key=lambda s: len(s), reverse=True)[:5]
    extracted_summary = ' '.join([s.text.strip() for s in extracted_sentences])

    # Adjust max and min lengths dynamically based on input size
    input_length = len(extracted_summary.split())
    max_length = min(512, input_length + 10)  # Allow larger summaries for longer texts
    min_length = int(0.2 * input_length)  # Ensure summary is not too short (at least 20% of input length)

    # Abstractive summarization on extracted content using beam search for better quality
    summary = summarizer(extracted_summary,
                         max_length=max_length,
                         min_length=min_length,
                         do_sample=False,
                         num_beams=4)  # Increase number of beams for better quality

    return summary[0]['summary_text']

# Step 5: Function to extract text from PDF
def extract_text_from_pdf(file):
    reader = PyPDF2.PdfReader(file)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + '\n'
    return text

# Step 6: Write the Streamlit app code dynamically
with open('app.py', 'w') as f:
    f.write('''
import spacy
from transformers import pipeline
import PyPDF2
import streamlit as st

# Load SpaCy model for English
nlp = spacy.load("en_core_web_sm")

# Initialize the summarization pipeline using a pre-trained model
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

# Function to summarize text using a hybrid approach with SpaCy
def hybrid_summarize(text):
    doc = nlp(text)
    sentences = list(doc.sents)

    if not sentences:
        return "No content to summarize."

    # Extractive summarization: Select the top 5 longest sentences
    extracted_sentences = sorted(sentences, key=lambda s: len(s), reverse=True)[:5]
    extracted_summary = ' '.join([s.text.strip() for s in extracted_sentences])

    # Abstractive summarization on extracted content
    input_length = len(extracted_summary.split())
    max_length = min(512, input_length + 10)
    min_length = int(0.2 * input_length)  # 20% of input length
    summary = summarizer(extracted_summary,
                         max_length=max_length,
                         min_length=min_length,
                         do_sample=False,
                         num_beams=4)  # Improved summarization quality

    return summary[0]['summary_text']

# Function to extract text from PDF
def extract_text_from_pdf(file):
    reader = PyPDF2.PdfReader(file)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + '\\n'
    return text

# Streamlit app setup
st.title("Abstract Text Summarizer")

# Choose input method (paste text or upload PDF)
option = st.radio("Choose an input method", ("Paste Text", "Upload PDF"))

# If user chooses to paste text
if option == "Paste Text":
    input_text = st.text_area("Enter the text you want to summarize:")
    if st.button("Summarize"):
        if input_text:
            with st.spinner('Generating summary...'):
                summary = hybrid_summarize(input_text)
            st.subheader("Summary")
            st.write(summary)

# If user chooses to upload a PDF
elif option == "Upload PDF":
    uploaded_file = st.file_uploader("Upload a PDF file", type="pdf")
    if uploaded_file is not None:
        pdf_text = extract_text_from_pdf(uploaded_file)
        st.subheader("PDF Content")
        st.write(pdf_text)
        if st.button("Summarize PDF"):
            with st.spinner('Generating summary...'):
                summary = hybrid_summarize(pdf_text)
            st.subheader("Summary")
            st.write(summary)
''')

# Step 7: Set up ngrok authentication
!ngrok authtoken '2q1TEP0sMns2s5a4qhkyuuKKGgU_FxQSL8YK1pSecg2zUwxH'

# Step 8: Run the Streamlit app in a subprocess
def run_streamlit():
    os.system('streamlit run app.py')

p = multiprocessing.Process(target=run_streamlit)
p.start()

# Step 9: Set up ngrok tunnel for external access
time.sleep(5)  # Give Streamlit time to start
try:
    # Correcting the ngrok connection for HTTP tunneling
    public_url = ngrok.connect(addr="8501", proto="http")
    print(f"Open this URL to view the app: {public_url}")
except Exception as e:
    print(f"Error connecting to ngrok: {e}")

# Step 10: Ensure Streamlit process stops after code execution ends
p.join()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cpu


Pipeline Initialized Successfully
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Open this URL to view the app: NgrokTunnel: "https://e6ae-35-197-14-235.ngrok-free.app" -> "http://localhost:8501"
